In [1]:
%load_ext lab_black
%cd ..

/home/shim/cev/dl/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from sklearn.model_selection import StratifiedKFold
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (
    AlbertForSequenceClassification,
    AlbertTokenizer,
    DebertaForSequenceClassification,
    DebertaTokenizer,
    SqueezeBertTokenizer,
    SqueezeBertForSequenceClassification,
    XLNetTokenizer,
    XLNetForSequenceClassification,
)

from datasets import load_test_data, load_train_data, MyDataset, load_train_total_data
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random

In [7]:
postfix = "distilbert-base-uncased-focal-AdamW-lr1e-05_1"

In [8]:
with open("config/distilbert-base-uncased-ver2.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [9]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased'),
 'debug': False,
 'seed': 20210425,
 'train': {'SAM': False,
  'folds': [3],
  'checkpoints': ['results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05-dsver2_3.pth'],
  'loss': {'name': 'focal', 'gamma': 2},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 10,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ori'),
  'ver': 2,
  'batch_size': 35,
  'num_workers': 8}}

In [10]:
deck1 = np.load(f"results/distilbert-base-uncased/{postfix}-deck1.npz")
deck2 = np.load(f"results/distilbert-base-uncased/{postfix}-deck2.npz")

In [11]:
list(deck1.keys())

['fcfeat', 'tlevel', 'fclevel', 'feat']

In [12]:
list(deck2.keys())

['fcfeat', 'fclevel', 'feat']

In [13]:
feat1 = torch.from_numpy(deck1["feat"])

In [14]:
feat2 = torch.from_numpy(deck2["feat"])

In [23]:
feat1.shape, feat2.shape

(torch.Size([472972, 768]), torch.Size([1418916, 768]))

In [48]:
class CosineSimilarity:
    def __init__(self, A, eps=1e-8):
        self.A = A
        self.Astar = A.pow(2).sum(dim=1).sqrt_()
        self.eps = eps

    def __call__(self, B):
        if B.dim() == 1:
            B = B.unsqueeze(0)

        Bstar = B.pow(2).sum(dim=1).sqrt_()
        up = (self.A * B).sum(dim=1)
        bottom = (self.Astar * Bstar).clamp(min=self.eps)
        dist = up / bottom
        return dist

In [55]:
cos = CosineSimilarity(feat1)

In [105]:
out = cos(feat1[472968])

In [106]:
out.topk(10)

torch.return_types.topk(
values=tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000]),
indices=tensor([472968, 374522, 410849,  70646, 129613, 245897, 259787,  59712,  84996,
        258134]))

In [103]:
tlevels[376681]

2

In [63]:
tlevels = deck1["tlevel"]

In [104]:
(tlevels == 1).nonzero()

(array([     4,      5,     10, ..., 472953, 472955, 472968]),)